### 343. Метро

Машинист Пётр управляет составом на кольцевой линии Омского метрополитена, станции которой пронумерованы по часовой стрелке от 1 до 10. В Омском метрополитене строго придерживаются расписания, и каждый день состав начинает движение со станции 1 ровно в 5:00 и заканчивает на станции 10 ровно в 23:30.

В метрополитене ходит всего один-единственный поезд. Это значит, что он посещает станции последовательно: сначала станцию 1, потом станцию 2, затем станцию 3, ……, станцию 10, а потом опять станцию 1 и так далее, пока не наступит 23:30 (в 23:30 поезд прибывает на 10-ю станцию и больше в этот день никуда не едет). Движение поезда неравномерное, но переезд с одной станции на другую занимает не менее одной минуты. Временем остановки следует пренебречь.

Однажды Петру захотелось узнать, сколько в среднем людей одновременно находилось в его составе в течение дня. Он вычисляет эту величину так.

Рассмотрим все перегоны spanspan между соседними станциями, и зададим каждый из них пятью числами:

(station_from,station_to,time_from,time_to,passengers)
station_to=(station_from(mod10))+1

Тогда среднее количество одновременно находившихся в составе людей вычисляется так:
(∑_span(time_to − time_from) ⋅ passengers) / ∑_span(time_to − time_from)

Вам дан лог событий вида (pid, action, station_num, time), означающих, что пассажир с номером pid в момент времени time на станции station_num либо вошел в состав (action = 'in'), либо вышел из него (action = 'out').

Пожалуйста, помогите Петру найти интересующее его значение.

In [1]:
import datetime

In [2]:
log = input()

 3 in 1 05:00 5 in 1 05:00 5 out 2 05:05 6 in 2 05:05 7 in 3 05:12 1 in 4 05:22 2 in 5 05:32 4 in 6 05:42 3 out 6 05:42 6 out 6 05:42 7 out 6 05:42 1 out 6 05:42 10 in 7 06:00 11 in 8 12:00 3 in 9 16:00 10 out 9 16:00 11 out 9 16:00 2 out 9 16:00 4 out 9 16:00 3 out 10 23:30


Формирую из переданного на ввод лога список списков.

In [3]:
records = log.split()
data = [records[i:i+4] for i in range(0, len(records), 4)]
data = sorted(data, key=lambda x: x[3])
data

[['3', 'in', '1', '05:00'],
 ['5', 'in', '1', '05:00'],
 ['5', 'out', '2', '05:05'],
 ['6', 'in', '2', '05:05'],
 ['7', 'in', '3', '05:12'],
 ['1', 'in', '4', '05:22'],
 ['2', 'in', '5', '05:32'],
 ['4', 'in', '6', '05:42'],
 ['3', 'out', '6', '05:42'],
 ['6', 'out', '6', '05:42'],
 ['7', 'out', '6', '05:42'],
 ['1', 'out', '6', '05:42'],
 ['10', 'in', '7', '06:00'],
 ['11', 'in', '8', '12:00'],
 ['3', 'in', '9', '16:00'],
 ['10', 'out', '9', '16:00'],
 ['11', 'out', '9', '16:00'],
 ['2', 'out', '9', '16:00'],
 ['4', 'out', '9', '16:00'],
 ['3', 'out', '10', '23:30']]

Создам счётчик, который будет считать входящих и выходящих пассажиров.

In [4]:
for i, record in enumerate(data):
    record[2] = int(record[2])
    record.append(1)

    if i != 0:
        if data[i][1] == 'in':
            data[i][4] = data[i - 1][4] + 1
        elif data[i][1] == 'out':
            data[i][4] = data[i - 1][4] - 1
data

[['3', 'in', 1, '05:00', 1],
 ['5', 'in', 1, '05:00', 2],
 ['5', 'out', 2, '05:05', 1],
 ['6', 'in', 2, '05:05', 2],
 ['7', 'in', 3, '05:12', 3],
 ['1', 'in', 4, '05:22', 4],
 ['2', 'in', 5, '05:32', 5],
 ['4', 'in', 6, '05:42', 6],
 ['3', 'out', 6, '05:42', 5],
 ['6', 'out', 6, '05:42', 4],
 ['7', 'out', 6, '05:42', 3],
 ['1', 'out', 6, '05:42', 2],
 ['10', 'in', 7, '06:00', 3],
 ['11', 'in', 8, '12:00', 4],
 ['3', 'in', 9, '16:00', 5],
 ['10', 'out', 9, '16:00', 4],
 ['11', 'out', 9, '16:00', 3],
 ['2', 'out', 9, '16:00', 2],
 ['4', 'out', 9, '16:00', 1],
 ['3', 'out', 10, '23:30', 0]]

Теперь из лога нужно откинуть дубли с одинаковым номером станции и временем, в которых отражена посадка-высадка пассажиров. Таким образом в логе останутся только временные промежутки между перегонами и конечное число пассажиров в вагоне, что и нужно для расчётов. 

In [5]:
last_occurrences = {} #Создается словарь для отслеживания последнего вхождения каждой станции

for record in reversed(data): #Итерирую по списку в обратном порядке
    station_num = record[2]
    time = record[3]
    key = (station_num, time) #Формирую уникальный ключ на основе номера станции и времени
    if key in last_occurrences: #Если ключ уже есть в словаре, то запись пропускается
        continue
    last_occurrences[key] = record #Иначе добавляется в словарь

data = list(last_occurrences.values()) #Преобразую словарь обратно в список
data.reverse()
data

[['5', 'in', 1, '05:00', 2],
 ['6', 'in', 2, '05:05', 2],
 ['7', 'in', 3, '05:12', 3],
 ['1', 'in', 4, '05:22', 4],
 ['2', 'in', 5, '05:32', 5],
 ['1', 'out', 6, '05:42', 2],
 ['10', 'in', 7, '06:00', 3],
 ['11', 'in', 8, '12:00', 4],
 ['4', 'out', 9, '16:00', 1],
 ['3', 'out', 10, '23:30', 0]]

Вычисляю разницу во времени между нахождением на станциях, тем самым получу длительность перегонов.

In [6]:
for i, record in enumerate(data):
    time = record[3]
    record[3] = datetime.datetime.strptime(time, '%H:%M')

    if i != 0:
        time_diff = record[3] - data[i - 1][3]
        time_diff_sec = time_diff.total_seconds() #Разность конвертируется в секунды
        data[i - 1].append(time_diff_sec) #И записывается в предыдущую строку

    if i == len(data) - 1:
        data[i].append(0) #В последнюю строку добавляется ноль, т.к. поезд прибыл на конечную станцию
data

[['5', 'in', 1, datetime.datetime(1900, 1, 1, 5, 0), 2, 300.0],
 ['6', 'in', 2, datetime.datetime(1900, 1, 1, 5, 5), 2, 420.0],
 ['7', 'in', 3, datetime.datetime(1900, 1, 1, 5, 12), 3, 600.0],
 ['1', 'in', 4, datetime.datetime(1900, 1, 1, 5, 22), 4, 600.0],
 ['2', 'in', 5, datetime.datetime(1900, 1, 1, 5, 32), 5, 600.0],
 ['1', 'out', 6, datetime.datetime(1900, 1, 1, 5, 42), 2, 1080.0],
 ['10', 'in', 7, datetime.datetime(1900, 1, 1, 6, 0), 3, 21600.0],
 ['11', 'in', 8, datetime.datetime(1900, 1, 1, 12, 0), 4, 14400.0],
 ['4', 'out', 9, datetime.datetime(1900, 1, 1, 16, 0), 1, 27000.0],
 ['3', 'out', 10, datetime.datetime(1900, 1, 1, 23, 30), 0, 0]]

Осталось только подставить все значения в указанную формулу.

In [7]:
total_time = 18 * 3600 + 30 * 60
total_product_sum = sum(record[-2] * record[-1] for record in data)
result = total_product_sum / total_time
print("{:.12f}".format(result))

2.405405405405
